# Contenido

1. **Carga de librerías y datos**  
   - 1.a. Librerías  
   - 1.b. Funciones  
   - 1.c. Carga de datos  

2. **Modelado**  
   - 2.a. Creación variables exógenas
   - 2.b. Entrenamiento del modelo
   - 2.c. Evaluación


# 1. Carga de librerías y datos

##  1.a. Librerías

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Modelado y Forecasting
# ==============================================================================
import xgboost
import lightgbm
import catboost
import sklearn
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

import skforecast
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
import shap

import cloudpickle
import pandas as pd
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import plotly.graph_objects as go

In [2]:
from Creacion_exog import calculo_variables_exogenas

## 1.b. Funciones

In [3]:
def train_val_test(datos, fin_train, fin_val):
    # Separación de datos en entrenamiento, validación y test
    # ==============================================================================
    datos_train = datos.loc[: fin_train]
    datos_val   = datos.loc[fin_train:fin_val]
    datos_test  = datos.loc[fin_val:]
    print(
        f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  "
        f"(n={len(datos_train)})"
    )
    print(
        f"Fechas validación : {datos_val.index.min()} --- {datos_val.index.max()}  "
        f"(n={len(datos_val)})"
    )
    print(
        f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  "
        f"(n={len(datos_test)})"
    )
    return datos_train, datos_val, datos_test

def imputar_nulos_por_hora(datos):
    datos.index = pd.to_datetime(datos.index)
    horas = datos.index.hour
    
    # Promedio por hora y sustitución el valores Nan
    media_por_hora = datos.groupby(horas).transform('mean')
    datos = datos.fillna(media_por_hora)
    
    return datos

def auxiliar(variables_exogenas):
    # Selección de variables exógenas a incluir en el modelo
    # ==============================================================================
    exog_cols = []
    # Columnas que terminan con _seno o _coseno son seleccionadas
    exog_cols.extend(variables_exogenas.filter(regex='_seno$|_coseno$').columns.tolist())
    
    # Columnas que empiezan con festivo_ son seleccionadas
    exog_cols.extend(variables_exogenas.filter(regex='^festivo_.*').columns.tolist())
    exog_cols.extend(['FESTIVO'])
    
    variables_exogenas = variables_exogenas.filter(exog_cols, axis=1)
    return exog_cols, variables_exogenas

def tratamiento(datos, variables_exogenas, fin_train, fin_val):
    # Combinar variables exógenas y target en el mismo dataframe
    # ==============================================================================
    datos = datos[['CANTIDAD']].merge(
        variables_exogenas,
        left_index=True,
        right_index=True,
        how='left'
    )
    
    # Debido a la creación de medias móviles, hay valores NaN al principio de la serie.
    # Debido a holiday_dia_siguiente hay valores NaN al final de la serie.
    # Las columnas numéricas se convierten a float32.
    datos = datos.dropna()
    datos = datos.astype({col: np.float32 for col in datos.select_dtypes("number").columns})
    datos_train = datos.loc[: fin_train, :].copy()
    datos_val   = datos.loc[fin_train:fin_val, :].copy()
    datos_test  = datos.loc[fin_val:, :].copy()
    return datos, datos_train, datos_val, datos_test

def prepare_time_series(data, column_name='CANTIDAD', freq='H'):
    data = data.to_frame(name=column_name)
    data.index = pd.to_datetime(data.index)
    data = data.asfreq(freq)
    data.index.name = 'FECHA'
    return data

## 1.c. Carga de datos


In [4]:
info = pd.read_csv('../../1-DATOS/3-DATOS DE RESULTADOS/ANALISIS DESCRIPTIVO/info_clusters.csv', index_col = 0) 

In [5]:
df = pd.read_parquet('../../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS_HORARIO_2022_2023_FILTRADO.parquet')

# 2. Modelado

In [6]:
# Transformación: codificación ordinal + conversión a tipo "category"
# ==============================================================================
pipeline_categorical = make_pipeline(
    OrdinalEncoder(
        dtype=int,
        handle_unknown="use_encoded_value",
        unknown_value=-1,
        encoded_missing_value=-1
    ),
    FunctionTransformer(
        func=lambda x: x.astype('category'),
        feature_names_out= 'one-to-one'
    )
)

transformer_exog = make_column_transformer(
    (
        pipeline_categorical,
        make_column_selector(dtype_include=['category', 'object']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

In [7]:
# Espacio de búsqueda de hiperparámetros
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1)
    } 
    return search_space

Los resultados mostrados en la siguiente celda corresponden a un ejemplo reducido, donde únicamente se han generado 3 modelos (ya que la ejecución completa implica un elevado coste computacional). Sin embargo, el código proporcionado reproduce la creación de todos los modelos predictivos utilizados en el proyecto. Para la realización del mismo, se ejecutó el código completo.

In [ ]:
import time
fin_train = '2023-04-30 23:59:00'
fin_val = '2023-09-30 23:59:00'

# Listas para almacenar las predicciones, métricas y modelos
metricas = []
tiempos_procesamiento = []
cluster = []

predicciones_dict = {}
datos_test_dict = {}

for columna in df.columns: # SI SE QUIERE EJEMPLO REDUCIDO, REDUCIR LA CANTIDAD DE COLUMNAS
    print(f'Procesando columna: {columna}')
    inicio_tiempo = time.time()
    # todo: LA GENERACIÓN DE VARIABLES EXÓGENAS SE DEBERÍA HACER ANTES, NO VARÍAN CON LA ESTACIÓN
    datos = df[columna]
    datos = prepare_time_series(datos, column_name='CANTIDAD', freq='H')
    datos = imputar_nulos_por_hora(datos)
    variables_exogenas = calculo_variables_exogenas(datos)
    exog_cols, variables_exogenas = auxiliar(variables_exogenas)
    datos, datos_train, datos_val, datos_test = tratamiento(datos, variables_exogenas, fin_train, fin_val)

    # Búsqueda de hiperparámetros
    # ==============================================================================
    forecaster = ForecasterAutoreg(
        regressor        = LGBMRegressor(random_state=15926, verbose=-1),
        lags             = 48,
        transformer_exog = transformer_exog,
        fit_kwargs       = {"categorical_feature": "auto"}
    )
    
    results_search, frozen_trial = bayesian_search_forecaster(
        forecaster         = forecaster,
        y                  = datos.loc[:fin_val, 'CANTIDAD'],
        exog               = datos.loc[:fin_val, exog_cols],
        search_space       = search_space,
        steps              = 8,
        refit              = False,
        metric             = 'mean_absolute_error',
        initial_train_size = len(datos_train),
        fixed_train_size   = False,
        n_trials           = 20,
        random_state       = 123,
        return_best        = True,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = True
    )

    # Backtesting en los datos de test incluyendo las variables exógenas
    # ==============================================================================
    metrica_LGBMRegressor, predicciones = backtesting_forecaster(
        forecaster         = forecaster,
        y                  = datos['CANTIDAD'],
        exog               = datos[exog_cols],
        steps              = 8,
        metric             = 'mean_absolute_error',
        initial_train_size = len(datos[:fin_val]),
        refit              = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = True
    )

    # Agregar la métrica a la lista de métricas
    metricas.append(metrica_LGBMRegressor['mean_absolute_error'].values[0])

    # Guardar predicciones y datos de prueba en diccionarios
    predicciones_dict[columna] = predicciones
    datos_test_dict[columna] = datos_test['CANTIDAD']
    
    # Guardar el cluster
    cluster.append(info.Cluster[info.station_id == columna].values[0])

    # Guardar el forecaster ajustado
    with open(f'../../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/MODELOS/forecaster_{columna}.pkl', 'wb') as f:
        cloudpickle.dump(forecaster, f)

    fin_tiempo = time.time()
    tiempo_empleado = fin_tiempo - inicio_tiempo
    tiempos_procesamiento.append(tiempo_empleado)

    print(f'El tiempo empleado para la columna {columna} ha sido: {tiempo_empleado:.2f} segundos')
    
# Convertir los diccionarios a DataFrames finales
predicciones_df_final = pd.DataFrame({key: value['pred'] for key, value in predicciones_dict.items()})
datos_test_df_final = pd.DataFrame(datos_test_dict)

# Crear el DataFrame con dos columnas
metricas_df = pd.DataFrame({
    'columna': df.columns,
    'metrica': metricas,
    'tiempo_procesamiento': tiempos_procesamiento,
    'cluster': cluster
})

predicciones_df_final.to_csv('../../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/predicciones_test.csv')
datos_test_df_final.to_csv('../../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/datos_test.csv')
metricas_df.to_csv('../../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/metricas.csv')

Procesando columna: 1


Best trial: 19. Best value: 4.02418: 100%|██████████| 20/20 [06:55<00:00, 20.77s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'n_estimators': 600, 'max_depth': 7, 'min_data_in_leaf': 466, 'learning_rate': 0.011027759675487456, 'feature_fraction': 0.9, 'max_bin': 150, 'reg_alpha': 0.9, 'reg_lambda': 0.5}
  Backtesting metric: 4.024178095800369



100%|██████████| 273/273 [00:09<00:00, 28.21it/s]


El tiempo empleado para la columna 1 ha sido: 436.52 segundos
Procesando columna: 2


...

El tiempo empleado para la columna 2 ha sido: 456.94 segundos
Procesando columna: 3


Best trial: 15. Best value: 3.44361: 100%|██████████| 20/20 [06:53<00:00, 20.66s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'n_estimators': 400, 'max_depth': 9, 'min_data_in_leaf': 319, 'learning_rate': 0.061510068002042555, 'feature_fraction': 0.6, 'max_bin': 200, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.2}
  Backtesting metric: 3.4436091166867513



100%|██████████| 273/273 [00:08<00:00, 31.00it/s]


El tiempo empleado para la columna 3 ha sido: 434.94 segundos
